In [28]:
import os
from pytubefix import YouTube
from pytubefix.cli import on_progress
import uuid

In [29]:
url = "https://www.youtube.com/shorts/4RMEJSpMMbw"

yt = YouTube(url, on_progress_callback = on_progress)
print(yt.title)
 
ys = yt.streams.get_highest_resolution()
video = ys.download()

늘 기회를 놓치는 사람의 특징


In [30]:
print(f"title: {yt.title}")
print(f"views : {yt.views}")
print(f"description: {yt.description}")
print(f"length: {yt.length}")

title: 늘 기회를 놓치는 사람의 특징
views : 133881
description: 
length: 59


In [31]:
# rename the mp4 to the title of the video without a file extension
unique_id = str(uuid.uuid4())
print(unique_id)
os.rename(video, unique_id)

de60447c-1169-462c-8ae3-59d12e7f68ad


In [32]:
# extract audio from the video
import time
import math
import ffmpeg

def extract_audio(input_file):
    extracted_audio = f"audio-{input_file}.wav"
    stream = ffmpeg.input(input_file)
    stream = ffmpeg.output(stream, extracted_audio)
    ffmpeg.run(stream, overwrite_output=True)
    return extracted_audio

In [33]:
audio_extract = extract_audio(unique_id)
audio_extract

'audio-de60447c-1169-462c-8ae3-59d12e7f68ad.wav'

In [7]:
# transcribe the audio
from faster_whisper import WhisperModel
def transcribe(audio):

    model = WhisperModel("medium", device="cpu")
    segments, info = model.transcribe(audio)
    language = info[0]
    print(f"transcription language: {language}")
    segments = list(segments) # this is where the transcribe happens

    for segment in segments:
        print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))

    return language, segments

c:\Users\dgh00\OneDrive\바탕 화면\whisper\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
%%time

language, segments = transcribe(audio_extract)

transcription language: ko
[0.00s -> 3.52s]  분명 내가 꿈꾸던 상황이 있었는데
[3.52s -> 6.28s]  현실이 그와 조화를 이루지 못할 때
[6.28s -> 7.92s]  이미 버스는 떠났어, 병신아
[7.92s -> 10.12s]  그니까 내가 잘해보라 했을 때 좀 잘하지
[10.12s -> 14.96s]  다시 말해, 인지부조화 상태에 빠지면 우리 인간은
[14.96s -> 17.20s]  근데 사실 나도 그렇게까지 진심은 아니었는데
[26.36s -> 29.28s]  아니, 내가 이사님한테 부탁해서 어떻게 잡은 오디션인데
[29.28s -> 31.40s]  도대체 왜 안 보겠다는 거야?
[31.40s -> 32.88s]  그냥 좀 지치네
[32.88s -> 34.88s]  제발 정신 좀 차려, 병신아
[34.88s -> 37.08s]  너는 언제까지 네 인생이 너 없이 흘러가는 걸
[37.08s -> 38.68s]  그렇게 구경만 하고 있을 건데?
[47.12s -> 49.00s]  저기요, 기사님 기사님 제발
CPU times: total: 3min 20s
Wall time: 57.1 s


In [9]:
for segment in segments:
        print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))

[0.00s -> 3.52s]  분명 내가 꿈꾸던 상황이 있었는데
[3.52s -> 6.28s]  현실이 그와 조화를 이루지 못할 때
[6.28s -> 7.92s]  이미 버스는 떠났어, 병신아
[7.92s -> 10.12s]  그니까 내가 잘해보라 했을 때 좀 잘하지
[10.12s -> 14.96s]  다시 말해, 인지부조화 상태에 빠지면 우리 인간은
[14.96s -> 17.20s]  근데 사실 나도 그렇게까지 진심은 아니었는데
[26.36s -> 29.28s]  아니, 내가 이사님한테 부탁해서 어떻게 잡은 오디션인데
[29.28s -> 31.40s]  도대체 왜 안 보겠다는 거야?
[31.40s -> 32.88s]  그냥 좀 지치네
[32.88s -> 34.88s]  제발 정신 좀 차려, 병신아
[34.88s -> 37.08s]  너는 언제까지 네 인생이 너 없이 흘러가는 걸
[37.08s -> 38.68s]  그렇게 구경만 하고 있을 건데?
[47.12s -> 49.00s]  저기요, 기사님 기사님 제발


In [10]:
# SubRip(SRT)
# Subtitle Index: 0, 1, 2
# Timecode: Start and end markers. HH:MM:SS,sss format.
# Text: The subtitle text.

In [11]:
# this is a helper function that takes time in seconds and
# converts it into HH:MM:SS,sss format for the SRT subtitle files.

def format_time_for_srt(seconds):
    hours = math.floor(seconds / 3600)
    seconds %= 3600
    minutes = math.floor(seconds / 60)
    seconds %= 60
    miliseconds = round((seconds - math.floor(seconds)) * 1000)
    seconds=math.floor(seconds)
    formatted_time = f"{hours :02d}:{minutes :02d}:{seconds :02d},{miliseconds :03d}"

    return formatted_time

In [12]:
def generate_subtitle_file(input_file, language, segments):
    subtitle_file = f"sub-{input_file}.{language}.srt"
    text = ""
    for index, segment in enumerate(segments):
        segment_start = format_time_for_srt(segment.start)
        segment_end = format_time_for_srt(segment.end)

        text += f"{str(index + 1)}\n"
        text += f"{segment_start} --> {segment_end}\n"
        text += f"{segment.text}\n\n"

    # UTF-8 인코딩으로 파일을 열기
    with open(subtitle_file, 'w', encoding='utf-8') as f:
        f.write(text)

    return subtitle_file

In [13]:
subtitle_file = generate_subtitle_file(unique_id, language, segments)

In [14]:
def add_subtitle_to_video(input_file, subtitle_file, subtitle_language):
    video_input_stream = ffmpeg.input(input_file)
    subtitle_input_stream = ffmpeg.input(subtitle_file)
    output_video = f"output-{input_file}-{subtitle_language}.mp4"
    subtitle_track_title = subtitle_file.replace(".srt", "")
    stream = ffmpeg.output(video_input_stream, output_video,
                           vf = f"subtitles='{subtitle_file}'")
    ffmpeg.run(stream, overwrite_output=True)

In [15]:
add_subtitle_to_video(unique_id, subtitle_file, language)

In [16]:
os.remove(f"audio-{unique_id}.wav")
os.remove(f"sub-{unique_id}.{language}.srt")
os.remove(f"{unique_id}")